
**Project Title: Building a Simple Information Retrieval System using BM25 and GPT-3 and evaluated in the CISI collection.**

>**Project Overview:**

>The aim of this project is to introduce students to the basics of Information Retrieval (IR) and the BM25 ranking algorithm. The sollution to this project will help us to select the students to attend the course on deep learning applied to search. The students will build a simple IR system that retrieves relevant documents from a pre-defined corpus based on the user's query. The students will also use the chatGPT to assist in all phases of the project. The coding will be done using Google Colab notebooks and the deliverables will be placed in github account.

>**Project Outcomes:**

> Upon completion of the project, students will have gained the following skills:

>Understanding of the BM25 ranking algorithm
Familiarity with Information Retrieval systems
>Use of GitHub to store the notebooks and documentation
Experience with chatGPT for help in find solutions and coding
Exposure to Google Colab notebooks for code execution and presentation.


## Download Files
> Download dos arquivos da coleção de teste CISI

In [125]:
!wget https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz

--2023-03-02 01:21:45--  https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz
Resolving ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)... 130.209.240.253
Connecting to ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)|130.209.240.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775144 (757K) [application/gzip]
Saving to: ‘cisi.tar.gz’

cisi.tar.gz         100%[===================>] 756.98K  1.67MB/s    in 0.4s    

2023-03-02 01:21:46 (1.67 MB/s) - ‘cisi.tar.gz’ saved [775144/775144]



In [126]:
!tar -xf cisi.tar.gz

## Imports & Utils
> Definições de funções usadas para manipulação dos dados da coleção CISI

In [145]:
import re
import os
import json
import pandas as pd


In [128]:
def load_data(path:str=""):
  """
  Reading CISI collection files and converting them to a list of strings
  """
  with open(path) as f:
        lines = ""
        for l in f.readlines():
            lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
        lines = lines.lstrip("\n").split("\n")
  return lines

def extract_docs(lines:list=None):
    """
    Processes the CISI.ALL file and returns a dictionary with the fields: 
    id (it is a numerical integer referring to the id of the document), 
    and document written in natural language.
    """
    doc_set = {}
    doc_id = ""
    doc_text = ""

    for l in lines:
        if l.startswith(".I"):
            doc_id = l.split(" ")[1].strip() 
        elif l.startswith(".X"):
            doc_set[doc_id] = doc_text.lstrip(" ")
            doc_id = ""
            doc_text = ""
        else:
            doc_text += re.sub(r"[^a-zA-Z]+", ' ', l.strip()[3:] + " ").strip() 


    print(f"Number of documents = {len(doc_set)}")
    print("Doc id: ", 1) 
    print("Doc content: ", doc_set["1"]) 
    return doc_set

def extract_queries(lines:list=None):
    """    
    Processes the CISI.QRY file and returns a dictionary with the fields: 
    id (it is a numerical integer referring to the id of the Query), 
    and query written in natural language.
    """
    qry_set = {}
    qry_id = ""
    for l in lines:
        if l.startswith(".I"):
            qry_id = l.split(" ")[1].strip() 
        elif l.startswith(".W"):
            qry_set[qry_id] = l.strip()[3:]
            qry_id = ""

    print(f"\n\nNumber of queries = {len(qry_set)}")
    print("Query id: ", 1) 
    print("Query content: ", qry_set["1"])     
    return qry_set

def extract_rel(path:str=""):
    """
    Reading and processing the CISI.REL file. 
    The function returns a dictionary with a key being the query id and a 
    list of relevant document ids as a value.
    """
    rel_set = {}
    with open(path) as f:
        for l in f.readlines():
            qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0] 
            doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

            if qry_id in rel_set:
                rel_set[qry_id].append(doc_id)
            else:
                rel_set[qry_id] = []
                rel_set[qry_id].append(doc_id)

    print(f"\n\nNumber of mappings = {len(rel_set)}")
    print("Ids of the relevant documents of the query 1", rel_set["1"])  
    return rel_set

def extract_files(path:str=""):
  """
  From a directory, extract the contents of the files:
  CISI.ALL, CISI.QRY and CISI.REL
  """
  # extract all documents
  doc_set = extract_docs(load_data(os.path.join(path, 'CISI.ALL')))

  #Extract all Queries
  qry_set = extract_queries(load_data(os.path.join(path, 'CISI.QRY')))

  # Extract all related doc and Queries
  rel_set = extract_rel(os.path.join(path, 'CISI.REL'))
  
  return doc_set, qry_set, rel_set

def save_docs_to_json(docs):
  """
  Converts the document set into a json mapping. 
  This mapping will be used by the pyserini implementation
  """
  lucene_dataset = pd.DataFrame(docs.items(),columns=['id','contents'], index=None)
  lucene_json = lucene_dataset.to_json(orient="records")
  luceneJsonObject = json.loads(lucene_json)
  json_object = json.dumps(luceneJsonObject, indent = 4) 
  with open('lucene.json', 'w') as outfile: 
      outfile.write(json_object)

In [129]:
#extract the contents
doc_set, qry, rel = extract_files('/content/')

Number of documents = 1460
Doc id:  1
Doc content:  18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


Number of queries = 112
Query id:  1
Query content:  What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?


Number of mappings = 76
Ids of the relevant documents o

In [130]:
save_docs_to_json(doc_set)

In [35]:
# Organizing the directories
!mkdir '/content/samples_to_bm25'
!mkdir '/content/output_bm25'
!mv '/content/lucene.json' '/content/samples_to_bm25/'

mkdir: cannot create directory ‘/content/samples_to_bm25’: File exists


#Indexing with BM25
O BM25 é um dos algoritmos mais usados para a construção de sistemas de IR. Ele é categorizado como uma recuperação esparsa e usa representações bag-of-words para a indexação dos documentos.

A implementação usada aqui é do Pyserini
>https://github.com/castorini/pyserini

## Install packages
> Instação Pyserini + rank-eval (usado para avaliação de sistemas IR) 

In [31]:
!pip install pyserini faiss intel-openmp nltk --quiet
!apt install libomp-dev
%cd /content
!rm -rf pygaggle && pip uninstall -y pygaggle
!git clone  --recursive https://github.com/castorini/pygaggle.git
%cd pygaggle
! pip install --editable . --quiet
! pip install gensim==4.2.0 jsonlines --quiet
! pip install faiss-cpu --no-cache --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 4.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp-10-dev libomp5-10
Suggested packages:
  libomp-10-doc
The following NEW packages will be installed:
  libomp-10-dev libomp-dev libomp5-10
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 351 kB of archives.
After this operation, 2,281 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libomp5-10 amd64 1:10.0.0-4ubuntu1 [300 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libomp-10-dev amd64 1:10.0.0-4ubuntu1 [47.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libomp-dev amd64 1:10.0-50~exp1 [2,824 B]
Fetc

In [70]:
!pip install rank-eval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Applying BM25 via lucene implementation

In [38]:
! python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/samples_to_bm25 \
  --index /content/output_bm25 \
  --generator DefaultLuceneDocumentGenerator \
  --threads 2 \
  --storePositions --storeDocvectors --storeRaw

2023-03-01 12:44:16,575 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-03-01 12:44:16,578 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-03-01 12:44:16,579 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-03-01 12:44:16,579 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: /content/samples_to_bm25
2023-03-01 12:44:16,580 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2023-03-01 12:44:16,584 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-03-01 12:44:16,585 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 2
2023-03-01 12:44:16,586 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: en
2023-03-01 12:44:16,586 INFO  [main] index.IndexCollectio

## Evaluate BM25
>Para isso iremos recuperar documentos para cada query e verificar a posição que o melhor documento é retornado.
> Metricas usadas: **MRR** e **nDCG**

A métrica **MRR** (Mean Reciprocal Rank) mede o quão bem um sistema de IR classifica as respostas mais relevantes em primeiro lugar.

O MRR é calculado dividindo 1 pelo rank da primeira resposta relevante.

O objetivo é obter um MRR mais próximo de 1, o que indica que o sistema de IR está colocando as respostas mais relevantes no topo da lista com mais frequência

Já a Métrica **nDGC** (normalized Discounted Cumulative Gain) mede o grau em que as respostas retornadas para uma consulta estão classificadas corretamente de acordo com sua relevância. Para fazer isso, ela considera a relevância das respostas e sua posição na lista de resultados.



In [189]:
from pyserini.search.lucene import LuceneSearcher
from pygaggle.rerank.base import hits_to_texts

def get_results(path_out_bm25='/content/output_bm25', qrys=None, k=100):
  searcher = LuceneSearcher(path_out_bm25)
  results = {}
  for key, value in qrys.items():
    results_found = [found.docid for found in searcher.search(value, k)]
    scores = [found.score for found in searcher.search(value, k)]
    if key in rel:
      results[key]  = {'founds':results_found, 'true':rel[key], 'scores':scores}
  return results



In [190]:
results = get_results(qrys=qry)
print("Total results: ", len(results))
print("Total docs ids: ", len(results['2']['founds']))

Total results:  76
Total docs ids:  100


In [155]:
from rank_eval import Qrels, Run, evaluate

#Configs vars
qrels = Qrels()
qrels.add_multi(q_ids=rel.keys(),
                doc_ids=[v for k, v in rel.items()],
                scores=[[1]*len(v) for k, v in rel.items()])
run = Run()
run.add_multi(
    q_ids=results.keys(),
    doc_ids=[results[k]["founds"] for k in results.keys()],
    scores=[results[k]["scores"] for k in results.keys()],
)



In [178]:
# Compute scores for multiple metrics at once
evaluate(qrels, run, ["mrr","ndcg@10"])

{'mrr': 0.612313359700076, 'ndcg@10': 0.360460986561388}

# Análise

> Nesta análise complementar, a ideia é verificar as queries que piores tiveram seus documentos ranqueados pelo BM25 e como trabalhos futuros tentar melhorar com o uso do ChatGPT3

In [168]:
best_ranks_qid = {}
for k in results.keys():
  ranks = []
  for d_rel in results[k]['true']:
    if d_rel in results[k]['founds']:
      ranks.append(results[k]['founds'].index(d_rel))
  ranks.sort()
  best_ranks_qid[k] = ranks[0]

#Queries that got the worst rank by BM25
sorted(best_ranks_qid.items(), key=lambda x:x[1], reverse=True)[0:5]

[('14', 96), ('101', 75), ('61', 43), ('8', 40), ('43', 30)]

Podemos ver que a Querie de id **14** e **101** foram as piores ranqueadas.

In [180]:
results['14']['true']

['45', '420', '890']

In [182]:
doc_set['45']

'Measuring the Reliability of Subject Classification by Men and Machines Borko, Harold Procedures for measuring the consistency of document classification are described.. Three subject specialists classified 997 abstracts of psychological reports into one of eleven categories.. These abstracts were also mechanically classified by a computer program using a factor score computational procedure.. Each abstract was scored for all categories and assigned to the one with the highest score.. The three manual classifications were compared with each other and with the mechanical classifications, and a series of contingency coefficients was computed.. The average reliability of manual classification procedures was equal to .870.. The correlation between automatic and manual classification was .766.. '

In [183]:
doc_set['420']

'The Automatic Creation of Literature Abstracts Luhn, H.P. Experts of technical papers and magazine articles that serve the purposes of conventional abstracts have been created entirely by automatic means.. In the exploratory research described, the complete text of an article in machine- readable form is scanned by an IBM 704 data-processing machine and analyzed in accordance with a standard program..  Statistical information derived from word frequency and distribution is used by the machine to compute a relative measure of significance, first for individual words and then for sentences.. Sentences scoring highest in significance are extracted and printed out to become the "auto-abstract".. '

In [184]:
doc_set['890']

'Pattern Recognition and Structure-Activity Relationship Studies.. Computer-Assisted Prediction of Antitumor Activity in Structurally Diverse Drugs in an Experimental Mouse Brain Tumor System Chu, K. C. Feldman, R. J. Shapiro, M. B. Hazard, G. F. Jr. Geran, R. I. This paper reports the application of pattern recognition and substructural analysis to the problem of predicting the antineoplastic activity of 24 test compounds in an experimental mouse brain tumor system based on 138 structurally diverse compounds tested in this tumor system.. The molecules were represented by three types of substructural fragments, the augmented atom, the heteropath, and the ring fragments.. Of the two pattern recognition methods used to predict the activity of the test compounds the nearest neighbor method predicted 83% correctly while the learning machine  method predicted 92% correctly.. The test structures and the important substructural fragments used in this study are given and the implications of th

In [191]:
print("Query 14:", qry['14'])
print("Top 1 and 2 by BM25")
print()
print(doc_set[results['14']['founds'][0]])
print('-----------------------')
print(doc_set[results['14']['founds'][1]])
print()



Query 14: What future is there for automatic medical diagnosis?
Top 1 and 2 by BM25

Libraries and Technological Forces Affecting Them Cuadra, C.A. 1.  What do we mean by technology and what kind of technological forces are we concerned with? 2.  Why is it important to be concerned with technology in thinking about the future of libraries? 3.  What kinds of technology are particularly important for libraries? 4.  How can this technology be applied today? 5.  What can we foresee for the future, as we move toward the year 2000? 6.  What, if anything, should we do tomorrow to try to get from here to here? 
-----------------------
Computer Indexing of Medical Articles - Project Medico Artandi, Susan An automatic indexing method is described in which index tags for documents are generated by the computer.. The computer scans the text of periodical articles and automatically assigns to them index terms with their respective weights on the basis of explicitly defined text characteristics.. A 

# Report + uso ChatGPT3

Neste trabalho foi desenvolvido um sistema simples de ranqueamento de documentos usado o algoritmo BM25, implementado via Pyserini.

A primeira etapa foi baixar a coleção de teste do CISI, extrai os documentos, querys e Qrels. A partir das informações mapeadas, foi indexado os documentos com o BM25 (LuceneSearcher) e logo depois foi avaliado a recuperação dos documentos.

Para a avaliação foi interado cada query sobre a indexação feita na etapa anterior, para essa etapa também foi usado o pyserini, mais especificamente a função "LuceneSearcher"e atribuido um k, esse valor de k consiste na quantidade de documentos retornados de resposta por query, por padrão foi definido 100.

Com a inferência feita em todas as querys foi  então computada a avaliação com as métricas MRR e nDCG. Como resultado vemos que em MRR foi obtido 61% enquando de nDCG igual a  0.36, os resultados apontam que é necessário melhoria no ranqueamento, a métrica MRR diz que a probabilidade do documento relevante estar próximo da primeira posição é de 61%.

**Uso com o ChatGPT**

Até o momento desse projeto o ChatGPT foi usado para entender melhor como o algoritmo do BM25 funciona; gerar explicações sobre as métricas MRR e nDCG e por fim foi tentado resumir alguns documentos, porém, não foi satisfatoria a saida via chatGPT.


### **Output ChatGPT3**

**Explain didactically how the BM25 works**

The BM25 algorithm is a ranking function that is commonly used in information retrieval systems to retrieve relevant documents in response to a user's query. Here's how it works:

Preprocessing: The first step is to preprocess the query and the document collection. This typically involves tokenization (breaking the text into individual words or terms), stopword removal (eliminating common words such as "the" and "and"), and stemming (reducing words to their base form, such as "running" to "run").

Term frequency: For each term in the query and document collection, the algorithm calculates the number of times the term appears in each document. This is known as the term frequency (TF).

Inverse document frequency: The algorithm also calculates the inverse document frequency (IDF) for each term. The IDF is a measure of how important a term is in the document collection. Terms that appear frequently in many documents (such as "the" or "and") have a low IDF, while terms that appear rarely in the document collection have a high IDF.

BM25 score: The BM25 score is calculated for each document in the collection based on its similarity to the query. The score is based on the TF and IDF values for each term in the query and the document. The formula for the BM25 score is:

score(D,Q) = sum(w(qi,D) * IDF(qi) * ((k1 + 1) * TF(qi,D)) / (k1 * (1 - b + b * (|D| / avgdl)) + TF(qi,D)))

score(D,Q) is the BM25 score for document D given query Q
w(qi,D) is a weighting factor that determines the importance of each query term
IDF(qi) is the IDF for query term qi
TF(qi,D) is the term frequency for term qi in document D
k1 and b are tuning parameters that control the impact of term frequency and document length on the score
|D| is the length of document D in terms
avgdl is the average document length in terms
The BM25 algorithm ranks documents based on their BM25 scores, with higher scores indicating a greater similarity to the query.

In summary, the BM25 algorithm calculates a score for each document in the collection based on the similarity of its terms to those in the query, taking into account the importance of each term and the length of the document. This allows it to effectively retrieve relevant documents in response to a user's query.

**Explain in a didactic way the MRR and nDCG metrics for information retrieval**

MRR (Mean Reciprocal Rank) and nDCG (Normalized Discounted Cumulative Gain) are evaluation metrics commonly used in information retrieval to measure the effectiveness of a system in returning relevant results to a user's query.

MRR is a measure of how quickly the relevant results are returned by the system. It is calculated as the average of the reciprocal ranks of the first relevant document for each query. In other words, for each query, the reciprocal rank of the first relevant document is calculated (i.e., 1/rank) and then the average of these reciprocal ranks is taken across all queries. The higher the MRR, the more quickly the relevant results are returned by the system.

nDCG is a measure of the quality of the results returned by the system. It takes into account both the relevance of the returned results and their rank in the list of results. It is calculated by first assigning a relevance score (usually a binary value) to each document based on how relevant it is to the query, and then calculating a cumulative gain score for the top k documents in the result list. The cumulative gain score is a measure of how much relevant information is contained in the top k documents, with greater weight given to documents that are higher in the result list. The cumulative gain score is then normalized by a discounted gain score, which gives greater weight to documents that are more relevant and appear higher in the result list. The nDCG score is the average of the normalized discounted cumulative gain scores across all queries. The higher the nDCG, the better the quality of the results returned by the system.

In summary, MRR measures how quickly the relevant results are returned by the system, while nDCG measures the quality of the results returned by the system. Both metrics are important for evaluating the effectiveness of an information retrieval system in returning relevant results to a user's query.


**Generate a summary** 

**Measuring the Reliability of Subject Classification by Men and Machines Borko, Harold Procedures for measuring the consistency of document classification are described.. Three subject specialists classified 997 abstracts of psychological reports into one of eleven categories.. These abstracts were also mechanically classified by a computer program using a factor score computational procedure.. Each abstract was scored for all categories and assigned to the one with the highest score.. The three manual classifications were compared with each other and with the mechanical classifications, and a series of contingency coefficients was computed.. The average reliability of manual classification procedures was equal to .870.. The correlation between automatic and manual classification was .766..**

> ChatGPT:

> In this study, the reliability of document classification was measured using both manual and automated methods. Three subject specialists manually classified 997 abstracts of psychological reports into one of eleven categories, while a computer program used a factor score computational procedure to mechanically classify the same abstracts. Each abstract was scored for all categories and assigned to the one with the highest score. The manual classifications were compared with each other and with the mechanical classifications, and contingency coefficients were computed. The results showed that the average reliability of manual classification was .870, while the correlation between automatic and manual classification was .766.